<a href="https://colab.research.google.com/github/ehas1/Statistical-Bias-in-ML/blob/main/Neural_Network_LIME_and_SHAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import shap
shap.initjs()
! pip install lime


In [ ]:
df_filtered = pd.read_csv('cox-violent-parsed_filt.csv')


In [ ]:
X = df_filtered[["sex", 'age', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree']]
Y = df_filtered['is_recid']

categorical_features = ['sex', 'race', 'c_charge_degree']
numerical_features = ['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count']

X_encoded = pd.get_dummies(X, columns=categorical_features)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import BatchNormalization
import joblib
import os

# Create directory for saved models if it doesn't exist
if not os.path.exists('recidivism_model'):
    os.makedirs('recidivism_model')

# Assume df_filtered is already loaded

# Define features and target
X = df_filtered[["sex", 'age', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree']]
Y = df_filtered['is_recid']

categorical_features = ['sex', 'race', 'c_charge_degree']
numerical_features = ['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count']

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=categorical_features)

# Normalize numerical features
scaler = StandardScaler()
X_encoded[numerical_features] = scaler.fit_transform(X_encoded[numerical_features])

# Convert Y to categorical (binary classification)
Y = Y.astype(int)  # Ensure it's integer (0 or 1)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=42)
y_train = y_train.clip(0, 1)

# Define Neural Network Model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu', kernel_regularizer=l2(0.02)),
    BatchNormalization(),
    Dropout(0.4),

    # Second block
    Dense(64, activation='relu', kernel_regularizer=l2(0.02)),
    BatchNormalization(),
    Dropout(0.3),

    # Third block
    Dense(32, activation='relu', kernel_regularizer=l2(0.02)),
    BatchNormalization(),
    Dropout(0.2),

    # Output layer
    Dense(1, activation='sigmoid', kernel_regularizer=l2(0.02))
])

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    mode='min'
)

# Learning rate reduction on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=0.00001,
    verbose=1
)

# Train Model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=64,
    callbacks=[early_stopping, reduce_lr]
)

# Extract training history
history_dict = history.history

# Evaluate Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Create plots for loss and accuracy to check for overfitting
plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(history_dict['loss'], label='Training Loss', linestyle='-', marker='o', color='blue')
plt.plot(history_dict['val_loss'], label='Validation Loss', linestyle='-', marker='o', color='red')
plt.xlabel('Epochs')
plt.ylabel('Loss (Binary Crossentropy)')
plt.title('Training vs Validation Loss')
plt.legend()
plt.grid(True)

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_dict['accuracy'], label='Training Accuracy', linestyle='-', marker='o', color='blue')
plt.plot(history_dict['val_accuracy'], label='Validation Accuracy', linestyle='-', marker='o', color='green')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training vs Validation Accuracy')
plt.legend()
plt.grid(True)

# Show plots
plt.tight_layout()
plt.show()

# Save the trained model
model.save('recidivism_model/neural_network_model.keras')
print("Model saved to 'recidivism_model/neural_network_model.keras'")

# Save the scaler for future preprocessing
joblib.dump(scaler, 'recidivism_model/scaler.save')
print("Scaler saved to 'recidivism_model/scaler.save'")

# Save feature names for reference
feature_names = {
    'categorical_features': categorical_features,
    'numerical_features': numerical_features
}
joblib.dump(feature_names, 'recidivism_model/feature_names.save')
print("Feature names saved to 'recidivism_model/feature_names.save'")

print("\nTo load the recidivism neural network model in another notebook:")
print("""
# Load the saved model and components
from tensorflow.keras.models import load_model
import joblib

model = load_model('recidivism_model/neural_network_model')
scaler = joblib.load('recidivism_model/scaler.save')
feature_names = joblib.load('recidivism_model/feature_names.save')
""")

#SHAP

##Individual Plots

In [ ]:
import shap
import numpy as np

# Convert background data to float explicitly
X_background = X_train.iloc[:50].astype(np.float64).values

# Define a prediction function that takes a 2D numpy array and returns predictions.
def f(X):
    return model.predict(X).flatten()

# Create the SHAP KernelExplainer using the numeric background data.
explainer = shap.KernelExplainer(f, X_background)

# Choose a sample index to explain (ensure this index exists in X_test).
sample_index = 299

# Get the sample as a 2D array and convert to float.
X_sample = X_test.iloc[[sample_index]].astype(np.float64).values

# Compute SHAP values for the chosen sample.
shap_values = explainer.shap_values(X_sample, nsamples=500)

# Initialize JS visualization (if using a Jupyter notebook)
shap.initjs()

# Display the force plot.
# Note: We're using the original DataFrame row (which has column names) for display.
shap.force_plot(explainer.expected_value, shap_values[0], X_test.iloc[sample_index])


##Global Plots

In [ ]:
import numpy as np
import shap
import numpy as np

# Initialize SHAP's JS visualization (run this once in the notebook)
shap.initjs()
# Select multiple samples (10 samples, for instance)
samples = X_test.iloc[:10]  # shape: (10, 540)
samples_array = samples.to_numpy().astype(np.float32)

# Compute SHAP values for each sample individually and store them in a list.
shap_values_list = []
for i in range(samples_array.shape[0]):
    sample_i = samples_array[i:i+1, :]  # shape (1, 540)
    # Compute SHAP values for the single sample.
    # DeepExplainer returns a list (one entry per model output), so we take the first element.
    shap_value_i = explainer.shap_values(sample_i)[0]

    # For a single sample we observed a shape of (540, 1) so we transpose it to get (1, 540)
    if shap_value_i.shape[0] == sample_i.shape[1]:
        shap_value_i = shap_value_i.T
    shap_values_list.append(shap_value_i)

# Stack the individual arrays to form one array for all samples.
# Expected shape: (10, 540)
shap_values_multi_corrected = np.vstack(shap_values_list)

print("Corrected SHAP values shape:", shap_values_multi_corrected.shape)
# Now create a summary plot
shap.summary_plot(shap_values_multi_corrected, samples)


#LIME

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt
import joblib
import os
import shap

# Create directory for saved models if it doesn't exist
if not os.path.exists('recidivism_model'):
    os.makedirs('recidivism_model')

# Check if model and components exist
model_path = 'recidivism_model/neural_network_model.keras'
scaler_path = 'recidivism_model/scaler.save'
feature_names_path = 'recidivism_model/feature_names.save'

if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}. Please run recidivism_neural_network.py first.")

if not os.path.exists(scaler_path):
    raise FileNotFoundError(f"Scaler file not found at {scaler_path}. Please run recidivism_neural_network.py first.")

if not os.path.exists(feature_names_path):
    raise FileNotFoundError(f"Feature names file not found at {feature_names_path}. Please run recidivism_neural_network.py first.")

# Load the saved model and components
model = load_model(model_path)
scaler = joblib.load(scaler_path)
feature_names = joblib.load(feature_names_path)

# Load and preprocess the data
data_path = 'cox-violent-parsed_filt.csv'
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data file not found at {data_path}. Please ensure the data file is in the correct location.")

df_filtered = pd.read_csv(data_path)

# Define features and target (same as in the original script)
X = df_filtered[["sex", 'age', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree']]
Y = df_filtered['is_recid']

categorical_features = feature_names['categorical_features']
numerical_features = feature_names['numerical_features']

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=categorical_features)

# Store original X for reference
X_test_original = X.copy()

# Normalize numerical features
X_encoded[numerical_features] = scaler.transform(X_encoded[numerical_features])

# Train/Test Split (use same random_state as original)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=42)

# --------------------------
# LIME Explanation Setup
# --------------------------
def predict_fn(data):
    data = data.astype(np.float32)
    preds = model.predict(data)  # shape (n_samples, 1)
    return np.hstack([1 - preds, preds])

explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.to_numpy(),
    feature_names=X_train.columns.tolist(),
    class_names=["Not Recid", "Recid"],
    mode="classification",
    discretize_continuous=False  # show actual scaled values for numeric features
)

# --------------------------
# Helper Functions for Mapping & Plotting
# --------------------------
def extract_feature_name(feature_str):
    """
    Extract the pure feature name from a LIME explanation string.
    E.g., "age <= 0.5" becomes "age", "sex_F = 1" becomes "sex_F".
    """
    if isinstance(feature_str, str):
        for delim in [' <= ', ' > ', ' = ']:
            if delim in feature_str:
                return feature_str.split(delim)[0]
        return feature_str
    return str(feature_str)

def get_feature_value(feature_str, instance_index):
    """
    For a given explanation feature, return the original instance value.
    For one-hot encoded features, map back to the original categorical value.
    For numeric features, return the raw (unscaled) numeric value from X_test_original.
    """
    pure_feat = extract_feature_name(feature_str)

    # Check if the feature is a one-hot encoded categorical column.
    for cat in categorical_features:
        prefix = cat + "_"
        if pure_feat.startswith(prefix):
            # active_cat is the category represented by this column.
            active_cat = pure_feat[len(prefix):]
            orig_val = X_test_original.loc[instance_index, cat]
            return f"{active_cat} (actual: {orig_val})"

    # Otherwise, if the pure feature is a numeric feature,
    # use the original unscaled value from X_test_original.
    if pure_feat in numerical_features:
        num_val = X_test_original.loc[instance_index, pure_feat]
        return f"{num_val}"

    # Otherwise, return the feature string itself.
    return feature_str

def plot_lime_explanation(exp_data, instance_index, title):
    """
    Plot a horizontal bar chart for the given LIME explanation.
    Customizes the label to show the instance's value:
      - For numeric features: the raw number.
      - For one-hot categorical features: the mapped original category.
    """
    # exp_data is a list of (feature, weight) tuples.
    sorted_exp = sorted(exp_data, key=lambda x: abs(x[1]), reverse=True)
    features = [f for f, w in sorted_exp]
    weights = [w for f, w in sorted_exp]

    # Build custom labels by appending the instance's value.
    custom_labels = [f"{f} (value: {get_feature_value(f, instance_index)})" for f in features]

    plt.figure(figsize=(8, 4))
    plt.barh(custom_labels, weights, color='skyblue')
    plt.xlabel("LIME Weight")
    plt.title(title)
    plt.gca().invert_yaxis()  # highest weight at the top
    plt.tight_layout()
    plt.show()

# --------------------------
# Generate LIME Explanations and Visuals for 5 Random Test Instances
# --------------------------
print("\nGenerating LIME explanations for 5 random test instances...")
instances = X_test.sample(5, random_state=42)
for idx, row in instances.iterrows():
    explanation = explainer.explain_instance(
        data_row=row.values,
        predict_fn=predict_fn,
        num_features=10
    )

    # Determine predicted label
    pred_probs = predict_fn(row.values.reshape(1, -1))
    pred_label = np.argmax(pred_probs[0])
    available_labels = list(explanation.as_map().keys())
    if pred_label not in available_labels:
        pred_label = available_labels[0]

    print(f"\n--- LIME Explanation for Test Instance Index: {idx} ---")
    exp_list = explanation.as_list(label=pred_label)
    for feature, weight in exp_list:
        print(f"{feature} => {weight:.4f}")

    # Plot custom bar chart with values mapped to the original data
    plot_lime_explanation(exp_list, idx, f"LIME Explanation for Test Instance {idx}")

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import lime.lime_tabular
import matplotlib.pyplot as plt
import seaborn as sns

# --------------------------
# Data Preparation
# --------------------------
# Assume df_filtered is already loaded

# Define features and target (original DataFrame)
X = df_filtered[["sex", "age", "race", "juv_fel_count", "juv_misd_count",
                 "juv_other_count", "priors_count", "c_charge_degree"]]
Y = df_filtered["is_recid"]

# Define which columns are categorical and numeric
categorical_features = ['sex', 'race', 'c_charge_degree']
numerical_features = ['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count']

# One-hot encode categorical features (this is what the model uses)
X_encoded = pd.get_dummies(X, columns=categorical_features)

# Normalize numeric features in the encoded DataFrame
scaler = StandardScaler()
X_encoded[numerical_features] = scaler.fit_transform(X_encoded[numerical_features])
X_encoded = X_encoded.astype(np.float32)
Y = Y.astype(int)

# Train/test split on the encoded data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=42)
y_train = y_train.clip(0, 1)

# Also, keep a copy of the original data (before one-hot encoding) for mapping values
X_test_original = X.loc[X_test.index]

# --------------------------
# Neural Network Model
# --------------------------
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dense(8, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=18, batch_size=32)
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# --------------------------
# LIME Explanation Setup
# --------------------------
def predict_fn(data):
    data = data.astype(np.float32)
    preds = model.predict(data)  # shape (n_samples, 1)
    return np.hstack([1 - preds, preds])

explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.to_numpy(),
    feature_names=X_train.columns.tolist(),
    class_names=["Not Recid", "Recid"],
    mode="classification",
    discretize_continuous=False  # show actual (scaled) numeric values
)

# --------------------------
# Helper Functions for Mapping & Plotting
# --------------------------
def extract_feature_name(feature_str):
    """
    Extract the pure feature name from a LIME explanation string.
    E.g., "age <= 0.5" becomes "age", "sex_F = 1" becomes "sex_F".
    """
    if isinstance(feature_str, str):
        for delim in [' <= ', ' > ', ' = ']:
            if delim in feature_str:
                return feature_str.split(delim)[0]
        return feature_str
    return str(feature_str)

def get_feature_value(feature_str, instance_index):
    """
    For a given explanation feature, return the original instance value.
    For one-hot encoded features, map back to the original categorical value.
    For numeric features, return the raw (unscaled) numeric value from X_test_original.
    """
    pure_feat = extract_feature_name(feature_str)

    # Check if the feature is a one-hot encoded categorical column.
    for cat in categorical_features:
        prefix = cat + "_"
        if pure_feat.startswith(prefix):
            # active_cat is the category represented by this column.
            active_cat = pure_feat[len(prefix):]
            orig_val = X_test_original.loc[instance_index, cat]
            return f"{active_cat} (actual: {orig_val})"

    # Otherwise, if the pure feature is a numeric feature,
    # use the original unscaled value from X_test_original.
    if pure_feat in numerical_features:
        num_val = X_test_original.loc[instance_index, pure_feat]
        return f"{num_val}"

    return feature_str

def plot_lime_explanation_with_values(exp_data, instance_index, title):
    """
    Plot a horizontal bar chart for the LIME explanation exp_data.
    Custom labels include the feature name and the actual dataset value.
    """
    # exp_data is a list of (feature, weight) tuples.
    sorted_exp = sorted(exp_data, key=lambda x: abs(x[1]), reverse=True)
    features = [f for f, w in sorted_exp]
    weights = [w for f, w in sorted_exp]

    # Build custom labels by appending the instance's value from X_test_original.
    custom_labels = [f"{f} (value: {get_feature_value(f, instance_index)})" for f in features]

    plt.figure(figsize=(10, 6))
    plt.barh(custom_labels, weights, color='skyblue')
    plt.xlabel("LIME Weight")
    plt.title(title)
    plt.gca().invert_yaxis()  # highest weight on top
    plt.tight_layout()
    plt.show()

# --------------------------
# Generate LIME Explanations and Visuals for 5 Random Test Instances
# --------------------------
instances = X_test.sample(5, random_state=42)
for idx, row in instances.iterrows():
    # Generate LIME explanation for this instance
    explanation = explainer.explain_instance(
        data_row=row.values,
        predict_fn=predict_fn,
        num_features=10
    )

    # Determine the predicted label
    pred_probs = predict_fn(row.values.reshape(1, -1))
    pred_label = np.argmax(pred_probs[0])
    available_labels = list(explanation.as_map().keys())
    if pred_label not in available_labels:
        pred_label = available_labels[0]

    print(f"\n--- LIME Explanation for Test Instance Index: {idx} ---")
    exp_list = explanation.as_list(label=pred_label)
    for feature, weight in exp_list:
        print(f"{feature} => {weight:.4f}")

    # Show the original LIME interactive visualization (if in Jupyter Notebook)
    explanation.show_in_notebook(show_table=True, show_all=False)

    # Plot custom bar chart with the actual dataset values in the labels
    plot_lime_explanation_with_values(exp_list, idx, f"LIME Explanation for Test Instance {idx}")
